In [97]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .appName("data_process") \
      .getOrCreate()

### Read File

In [98]:
csv_file = spark.read.csv("111年傷亡道路交通事故資料/111年度A2交通事故資料_2.csv", header = True) 
csv_file.show(5)

+--------+--------+--------+--------+------------+------------------+----------------------------------+--------+--------------------------------+-----------------------+--------------+------------------+------------------+------------------+------------------------+---------------------+---------------------+---------------------+-------------------+---------------------+-----------------+-----------------+-----------------+-------------------------------+-------------------------------+--------------------------------------------+------------------------------------+----------------------------------+------------------------+------------------------+-----------------------+-----------------------+------------+----------+-------------------------------+-------------------------------+------------------+--------------------+----------------------------------+------------------------------------+------------------------+------------------------+---------------------------+--------------

### Drop unnecessary columns and rows

In [99]:
cols = ("發生年度","發生月份")
clean_file = csv_file. \
                drop(*cols). \
                filter(csv_file["道路類別-第1當事者-名稱"] != "國道")
print("Totol count:", csv_file.count(), "\n[國道] count:", csv_file.count() - clean_file.count())
clean_file.show(5)

Totol count: 59456 
[國道] count: 890
+--------+--------+------------+------------------+----------------------------------+--------+--------------------------------+-----------------------+--------------+------------------+------------------+------------------+------------------------+---------------------+---------------------+---------------------+-------------------+---------------------+-----------------+-----------------+-----------------+-------------------------------+-------------------------------+--------------------------------------------+------------------------------------+----------------------------------+------------------------+------------------------+-----------------------+-----------------------+------------+----------+-------------------------------+-------------------------------+------------------+--------------------+----------------------------------+------------------------------------+------------------------+------------------------+------------------------

### Fix the column of "Gender" ,add new colume "Object", fix column "發生時間"

In [100]:
clean_file.groupBy("當事者屬-性-別名稱").count().show()

+--------------------+-----+
|  當事者屬-性-別名稱|count|
+--------------------+-----+
|                  男|34063|
|    肇事逃逸尚未查獲|  267|
|無或物(動物、堆置物)| 3149|
|                  女|21087|
+--------------------+-----+



In [101]:
row_list = clean_file.collect()
dict_list = []
for row in row_list:
    temp = row.asDict()
    temp["無或物(動物、堆置物)"] = False
    temp["發生時間"] = temp["發生時間"][:2]
    dict_list.append(temp)


for i in range(len(dict_list)):
    if dict_list[i]["當事者屬-性-別名稱"] == "無或物(動物、堆置物)":
        count = int(dict_list[i]["當事者順位"])-1 #有幾個row要加欄位
        while count == 1:
            dict_list[i-count]["無或物(動物、堆置物)"] = True
            count -= 1


cols = clean_file.columns
cols.insert(34 , "無或物(動物、堆置物)")
df = spark.createDataFrame(dict_list).select(cols)

# rename column to "Gender" and drop rows of "當事者屬-性-別名稱" != "無或物(動物、堆置物)"
df = df.filter(df["當事者屬-性-別名稱"] != "無或物(動物、堆置物)").withColumnRenamed("當事者屬-性-別名稱","性別")

# validate
df.groupBy("性別").count().show()

df.show(10)


23/05/04 15:14:00 WARN TaskSetManager: Stage 280 contains a task of very large size (5044 KiB). The maximum recommended task size is 1000 KiB.


+----------------+-----+
|            性別|count|
+----------------+-----+
|              男|34063|
|肇事逃逸尚未查獲|  267|
|              女|21087|
+----------------+-----+

23/05/04 15:14:01 WARN TaskSetManager: Stage 283 contains a task of very large size (5044 KiB). The maximum recommended task size is 1000 KiB.


23/05/04 15:14:05 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 283 (TID 493): Attempting to kill Python Worker
+--------+--------+------------+------------------+----------------------------------+--------+--------------------------------+-----------------------+--------------+------------------+------------------+------------------+------------------------+---------------------+---------------------+---------------------+-------------------+---------------------+-----------------+-----------------+-----------------+-------------------------------+-------------------------------+--------------------------------------------+------------------------------------+----------------------------------+------------------------+------------------------+-----------------------+-----------------------+------------+----------+-------------------------------+-------------------------------+--------------------+----+--------------------+----------------------------------+--

### Split 死亡 and 受傷

In [102]:
from pyspark.sql.functions import split
import pyspark.sql.functions as f

df = df.withColumn('死亡人數', split(df['死亡受傷人數'], ';').getItem(0)) \
       .withColumn('受傷人數', split(df['死亡受傷人數'], ';').getItem(1)) \
       .withColumn("死亡人數", f.regexp_replace(f.col("死亡人數"), "[死亡]", "")) \
       .withColumn("受傷人數", f.regexp_replace(f.col("受傷人數"), "[受傷]", ""))


df.select('死亡人數', '受傷人數').show()

23/05/04 15:14:05 WARN TaskSetManager: Stage 284 contains a task of very large size (5044 KiB). The maximum recommended task size is 1000 KiB.


23/05/04 15:14:09 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 284 (TID 494): Attempting to kill Python Worker
+--------+--------+
|死亡人數|受傷人數|
+--------+--------+
|       0|       1|
|       0|       1|
|       0|       1|
|       0|       1|
|       0|       2|
|       0|       2|
|       0|       1|
|       0|       1|
|       0|       1|
|       0|       1|
|       0|       1|
|       0|       1|
|       0|       1|
|       0|       1|
|       0|       1|
|       0|       1|
|       0|       1|
|       0|       5|
|       0|       5|
|       0|       5|
+--------+--------+
only showing top 20 rows



### Extract 發生地點

In [103]:
df=df.withColumn('site_id', df.發生地點.substr(1, 6))

### Combine with calendar

In [104]:
calendar=spark.read.csv('111年中華民國政府行政機關辦公日曆表.csv',header=True, encoding='utf8')

calendar = calendar.drop(calendar.columns[1],calendar.columns[3]) \
                .withColumnRenamed(calendar.columns[0],"Date") \
                .withColumnRenamed(calendar.columns[2],"holiday")

df = df.join(calendar, df.發生日期 ==  calendar.Date, "inner") \
        .drop(calendar.Date)

df.show(10)

23/05/04 15:14:09 WARN TaskSetManager: Stage 287 contains a task of very large size (5044 KiB). The maximum recommended task size is 1000 KiB.


23/05/04 15:14:13 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 287 (TID 497): Attempting to kill Python Worker
+--------+--------+------------+------------------+----------------------------------+--------+--------------------------------+-----------------------+--------------+------------------+------------------+------------------+------------------------+---------------------+---------------------+---------------------+-------------------+---------------------+-----------------+-----------------+-----------------+-------------------------------+-------------------------------+--------------------------------------------+------------------------------------+----------------------------------+------------------------+------------------------+-----------------------+-----------------------+------------+----------+-------------------------------+-------------------------------+--------------------+----+--------------------+----------------------------------+--